In [1]:
import pandas as pd
import numpy as np

In [ ]:
gini = pd.read_csv('API_SI.POV.GINI_DS2_en_csv_v2_2728.csv')
gini.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa Western and Central,AFW,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,AGO,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,51.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Drop the 'Unnamed: 69' column if it exists and is irrelevant
if 'Unnamed: 69' in gini.columns:
    gini = gini.drop(columns=['Unnamed: 69'])

# Identify the columns that represent years (they are numeric strings)
year_columns = [col for col in gini.columns if col.isdigit() ]

# Identify the identifier columns
id_vars = [col for col in gini.columns if col not in year_columns]

# Melt the DataFrame to transform year columns into rows
gini_tidy = gini.melt(id_vars=id_vars, value_vars=year_columns, var_name='year', value_name='gini_value')

# Convert 'year' column to integer type
gini_tidy['year'] = pd.to_numeric(gini_tidy['year'])

# Display the head of the tidied DataFrame
display(gini_tidy.head())

,Country Name,Country Code,Indicator Name,Indicator Code,year,gini_value
0,Aruba,ABW,Gini index,SI.POV.GINI,1960,NaN
1,Africa Eastern and Southern,AFE,Gini index,SI.POV.GINI,1960,NaN
2,Afghanistan,AFG,Gini index,SI.POV.GINI,1960,NaN
3,Africa Western and Central,AFW,Gini index,SI.POV.GINI,1960,NaN
4,Angola,AGO,Gini index,SI.POV.GINI,1960,NaN


The `gini_tidy` DataFrame now has a 'year' column, making it easier to analyze time-series data for Gini index values.

In [ ]:
# Filter out rows with NaN gini_value
gini_no_nan = gini_tidy.dropna(subset=['gini_value'])

# Find the highest year for each country and use it to filter
gini_plot_data = gini_no_nan.loc[gini_no_nan.groupby('Country Name')['year'].idxmax()]

print(f"Data for the highest available year per country:")
print(gini_plot_data.shape)
gini_plot_data.head()

Data for the highest available year per country:
(171, 6)


,Country Name,Country Code,Indicator Name,Indicator Code,year,gini_value
15965,Albania,ALB,Gini index,SI.POV.GINI,2020,29.4
13626,Algeria,DZA,Gini index,SI.POV.GINI,2011,27.6
15432,Angola,AGO,Gini index,SI.POV.GINI,2018,51.3
17033,Argentina,ARG,Gini index,SI.POV.GINI,2024,42.4
16768,Armenia,ARM,Gini index,SI.POV.GINI,2023,27.2


In [ ]:
gini_plot_data.gini_value.min(), gini_plot_data.gini_value.max()

(23.8, 63.0)

In [ ]:
import plotly.express as px

# Create the choropleth map for gini value of country
fig = px.choropleth(
    gini_plot_data,
    locations='Country Code',
    color='gini_value',
    hover_name='Country Name',
    color_continuous_scale='spectral_r', # Changed colorscale to blue-to-red
    range_color=[20, 60],
    title='LatestGlobal Gini Index (Blue to Red: 20 to 60)',
    projection='natural earth'
)

fig.show()

In [4]:
from pandas_datareader import data, wb

start_year = 1990
end_year = 2022
countries = ['USA','IND','CHN','BRA','DEU']  # ISO3 codes accepted
# Indicators dictionary
indicators = {
    'NY.GDP.PCAP.KD': 'gdp_percap_const',
    'SL.UEM.TOTL.ZS': 'unemployment_pct',
    'FP.CPI.TOTL.ZG': 'inflation_pct',
    'SE.XPD.TOTL.GD.ZS': 'edu_spend_pct_gdp',
    'NE.TRD.GNFS.ZS': 'trade_pct_gdp',
    'SP.POP.TOTL': 'population'
}

from functools import reduce
frames = []
for code, name in indicators.items():
    print('Downloading', code)
    try:
        # Use wb.download, which is more robust for World Bank data and accepts 'country'
        df = wb.download(indicator=code, country=countries, start=start_year, end=end_year)
        df = df.reset_index() # country and year become columns

        # Rename columns: the indicator code to its friendly name, and 'country' to 'country_name'
        df = df.rename(columns={'country': 'country_name', code: name})

        frames.append(df[['country_name','year', name]])
    except Exception as e:
        print('Failed to fetch', code, ' — ', e)

if frames:
    data = reduce(lambda left, right: pd.merge(left, right, on=['country_name','year'], how='outer'), frames)
else:
    raise RuntimeError('No data frames fetched. Please download CSVs manually.')

data.sort_values(['country_name','year'], inplace=True)

/tmp/ipython-input-1333013610.py:22: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = wb.download(indicator=code, country=countries, start=start_year, end=end_year)


/tmp/ipython-input-1333013610.py:22: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = wb.download(indicator=code, country=countries, start=start_year, end=end_year)


/tmp/ipython-input-1333013610.py:22: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = wb.download(indicator=code, country=countries, start=start_year, end=end_year)
/tmp/ipython-input-1333013610.py:22: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = wb.download(indicator=code, country=countries, start=start_year, end=end_year)


/tmp/ipython-input-1333013610.py:22: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = wb.download(indicator=code, country=countries, start=start_year, end=end_year)


/tmp/ipython-input-1333013610.py:22: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = wb.download(indicator=code, country=countries, start=start_year, end=end_year)


In [10]:
print(data.shape)
data.head()

(165, 8)


,country_name,year,gdp_percap_const,unemployment_pct,inflation_pct,edu_spend_pct_gdp,trade_pct_gdp,population
0,Brazil,1990,6149.867889,NaN,2947.732772,NaN,15.155599,149143223
1,Brazil,1991,6107.648837,6.894,432.786662,NaN,16.592078,151724256
2,Brazil,1992,5973.982856,7.086,951.962053,NaN,19.253375,154275079
3,Brazil,1993,6167.460922,6.030,1927.380790,NaN,19.599321,156794577
4,Brazil,1994,6427.169019,6.578,2075.888398,NaN,18.674760,159265006


---
# Test Brazil Data

In [42]:
ind_df = pd.read_csv("WID_data_IN.csv", sep=";")
bzl_df = pd.read_csv("WID_data_BR.csv", sep=";")
usa_df = pd.read_csv("WID_data_US.csv", sep=";")
chn_df = pd.read_csv("WID_data_CN.csv", sep=";")
nld_df = pd.read_csv("WID_data_NL.csv", sep=";")
saf_df = pd.read_csv("WID_data_ZA.csv", sep=";")

print(f"Shape of India DataFrame: {ind_df.shape}")
print(f"Shape of Brazil DataFrame: {bzl_df.shape}")
print(f"Shape of USA DataFrame: {usa_df.shape}")
print(f"Shape of China DataFrame: {chn_df.shape}")
print(f"Shape of Netherlands DataFrame: {nld_df.shape}")
print(f"Shape of South Africa DataFrame: {saf_df.shape}")

Shape of India DataFrame: (734991, 7)
Shape of Brazil DataFrame: (543551, 7)
Shape of USA DataFrame: (1215596, 7)
Shape of China DataFrame: (580914, 7)
Shape of Netherlands DataFrame: (757711, 7)
Shape of South Africa DataFrame: (930128, 7)


WID variable codes follow a structured format:

- **s** → Share (the variable measures a share of income/wealth).
- **ptinc** → Pretax income (income before taxes and transfers).
- **j** → Equal-split adults (income is split equally between partners in a household).
- **992 vs. 999** → Age group codes:
    - 992 = Adults only (age 20+)
    - 999 = All ages (entire population, including children)

In [43]:
ind_df['variable'].unique()
ind_df_filtered = ind_df[ind_df['variable'].str.lower().str.startswith("sptincj999")].reset_index(drop=True)
bzl_df_filtered = bzl_df[bzl_df['variable'].str.lower().str.startswith("sptincj999")].reset_index(drop=True)
usa_df_filtered = usa_df[usa_df['variable'].str.lower().str.startswith("sptincj999")].reset_index(drop=True)
chn_df_filtered = chn_df[chn_df['variable'].str.lower().str.startswith("sptincj999")].reset_index(drop=True)
nld_df_filtered = nld_df[nld_df['variable'].str.lower().str.startswith("sptincj999")].reset_index(drop=True)
saf_df_filtered = saf_df[saf_df['variable'].str.lower().str.startswith("sptincj999")].reset_index(drop=True)

print(f"Filtered Shape of India DataFrame: {ind_df_filtered.shape}")
print(f"Filtered Shape of Brazil DataFrame: {bzl_df_filtered.shape}")
print(f"Filtered Shape of USA DataFrame: {usa_df_filtered.shape}")
print(f"Filtered Shape of China DataFrame: {chn_df_filtered.shape}")
print(f"Filtered Shape of Netherlands DataFrame: {nld_df_filtered.shape}")
print(f"Filtered Shape of South Africa DataFrame: {saf_df_filtered.shape}")
df_combined = pd.concat([ind_df_filtered, bzl_df_filtered, usa_df_filtered, chn_df_filtered, nld_df_filtered, saf_df_filtered], ignore_index=True)
print(f"Combined DataFrame Shape: {df_combined.shape}")
print(f"Sample data from combined DataFrame:\n{df_combined.head(5)}")

Filtered Shape of India DataFrame: (32141, 7)
Filtered Shape of Brazil DataFrame: (21686, 7)
Filtered Shape of USA DataFrame: (45289, 7)
Filtered Shape of China DataFrame: (22457, 7)
Filtered Shape of Netherlands DataFrame: (21672, 7)
Filtered Shape of South Africa DataFrame: (21692, 7)
Combined DataFrame Shape: (164937, 7)
Sample data from combined DataFrame:
  country    variable percentile  year   value  age pop
0      IN  sptincj999     p4p100  1820  0.9911  999   j
1      IN  sptincj999     p4p100  1850  0.9922  999   j
2      IN  sptincj999     p4p100  1880  0.9982  999   j
3      IN  sptincj999     p4p100  1900  0.9982  999   j
4      IN  sptincj999     p4p100  1910  0.9919  999   j


In [44]:
df_combined[['p_start', 'p_end']] = df_combined['percentile'].str.extract(r'p(\d+\.?\d*)p(\d+\.?\d*)').astype("float")
df_combined.head()

,country,variable,percentile,year,value,age,pop,p_start,p_end
0,IN,sptincj999,p4p100,1820,0.9911,999,j,4.0,100.0
1,IN,sptincj999,p4p100,1850,0.9922,999,j,4.0,100.0
2,IN,sptincj999,p4p100,1880,0.9982,999,j,4.0,100.0
3,IN,sptincj999,p4p100,1900,0.9982,999,j,4.0,100.0
4,IN,sptincj999,p4p100,1910,0.9919,999,j,4.0,100.0


In [45]:
# Filtering top 10%, top 1%, top 0.1%, and bottom 50%
top_10 = df_combined[(df_combined['p_start'] == 90) & (df_combined['p_end'] == 100)].reset_index(drop=True)
top_1 = df_combined[(df_combined['p_start'] == 99) & (df_combined['p_end'] == 100)].reset_index(drop=True)
top_0_1 = df_combined[(df_combined['p_start'] == 99.9) & (df_combined['p_end'] == 100)].reset_index(drop=True)
bottom_50 = df_combined[(df_combined['p_start'] == 0) & (df_combined['p_end'] == 50)].reset_index(drop=True)

df_income_summary = pd.concat([top_10, top_1, top_0_1, bottom_50], ignore_index=True)
df_income_summary['income_distribution'] = np.where(df_income_summary['p_start'] == 90, 'top_10%',
                                        np.where(df_income_summary['p_start'] == 99, 'top_1%',
                                        np.where(df_income_summary['p_start'] == 99.9, 'top_0.1%',
                                        np.where(df_income_summary['p_start'] == 0, 'bottom_50%', 'other'))))
df_income_summary

,country,variable,percentile,year,value,age,pop,p_start,p_end,income_distribution
0,IN,sptincj999,p90p100,1820,0.4800,999,j,90.0,100.0,top_10%
1,IN,sptincj999,p90p100,1850,0.5000,999,j,90.0,100.0,top_10%
2,IN,sptincj999,p90p100,1880,0.5460,999,j,90.0,100.0,top_10%
3,IN,sptincj999,p90p100,1900,0.5376,999,j,90.0,100.0,top_10%
4,IN,sptincj999,p90p100,1910,0.4480,999,j,90.0,100.0,top_10%
...,...,...,...,...,...,...,...,...,...,...
1699,ZA,sptincj999,p0p50,2020,0.0627,999,j,0.0,50.0,bottom_50%
1700,ZA,sptincj999,p0p50,2021,0.0627,999,j,0.0,50.0,bottom_50%
1701,ZA,sptincj999,p0p50,2022,0.0627,999,j,0.0,50.0,bottom_50%
1702,ZA,sptincj999,p0p50,2023,0.0627,999,j,0.0,50.0,bottom_50%


In [52]:
df_wide = df_income_summary.pivot_table(
    index=['country','year'],
    columns='income_distribution',
    values='value'
).reset_index(drop=False)
df_wide

income_distribution,country,year,bottom_50%,top_0.1%,top_1%,top_10%
0,BR,1820,0.1173,0.0970,0.2348,0.5324
1,BR,1850,0.1159,0.0977,0.2373,0.5380
2,BR,1880,0.1145,0.0984,0.2397,0.5436
3,BR,1900,0.1131,0.0990,0.2422,0.5492
4,BR,1910,0.1103,0.1003,0.2472,0.5605
...,...,...,...,...,...,...
421,ZA,2020,0.0627,0.0574,0.1921,0.6508
422,ZA,2021,0.0627,0.0574,0.1921,0.6508
423,ZA,2022,0.0627,0.0574,0.1921,0.6508
424,ZA,2023,0.0627,0.0574,0.1921,0.6508


In [54]:
# we will only be calculating income share : sptinc999 = share of national income for all ages
brazil_inequality = df_wide[df_wide['country'] == 'BR']
print(brazil_inequality.shape)
brazil_inequality.head()

(56, 6)


income_distribution,country,year,bottom_50%,top_0.1%,top_1%,top_10%
0,BR,1820,0.1173,0.0970,0.2348,0.5324
1,BR,1850,0.1159,0.0977,0.2373,0.5380
2,BR,1880,0.1145,0.0984,0.2397,0.5436
3,BR,1900,0.1131,0.0990,0.2422,0.5492
4,BR,1910,0.1103,0.1003,0.2472,0.5605


In [55]:
# convert to appropriate type and set proper country name
brazil_inequality['year'] = pd.to_datetime(brazil_inequality['year'], format='%Y').dt.year
brazil_inequality['bottom_50%'] = pd.to_numeric(brazil_inequality['bottom_50%'])
brazil_inequality['top_10%'] = pd.to_numeric(brazil_inequality['top_10%'])
brazil_inequality['country'] = brazil_inequality['country'].replace('BR', 'Brazil')

brazil_inequality.head()

/tmp/ipython-input-744908653.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brazil_inequality['year'] = pd.to_datetime(brazil_inequality['year'], format='%Y').dt.year
/tmp/ipython-input-744908653.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brazil_inequality['bottom_50%'] = pd.to_numeric(brazil_inequality['bottom_50%'])
/tmp/ipython-input-744908653.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

income_distribution,country,year,bottom_50%,top_0.1%,top_1%,top_10%
0,Brazil,1820,0.1173,0.0970,0.2348,0.5324
1,Brazil,1850,0.1159,0.0977,0.2373,0.5380
2,Brazil,1880,0.1145,0.0984,0.2397,0.5436
3,Brazil,1900,0.1131,0.0990,0.2422,0.5492
4,Brazil,1910,0.1103,0.1003,0.2472,0.5605


In [59]:
brazil_inequality = brazil_inequality[brazil_inequality['year'] >= 1990]
len(brazil_inequality)

35

In [61]:
brazil_inequality.head()

income_distribution,country,year,bottom_50%,top_0.1%,top_1%,top_10%
21,Brazil,1990,0.0912,0.1279,0.2724,0.5861
22,Brazil,1991,0.0912,0.1279,0.2724,0.5861
23,Brazil,1992,0.0912,0.1279,0.2724,0.5861
24,Brazil,1993,0.0912,0.1279,0.2724,0.5861
25,Brazil,1994,0.0912,0.1279,0.2724,0.5861


In [62]:
brazil_indicators = data[data['country_name']== 'Brazil']
brazil_indicators.year = pd.to_numeric(brazil_indicators['year'])
print(brazil_indicators.shape)
brazil_indicators.head()

(33, 8)


/tmp/ipython-input-2507986632.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brazil_indicators.year = pd.to_numeric(brazil_indicators['year'])


,country_name,year,gdp_percap_const,unemployment_pct,inflation_pct,edu_spend_pct_gdp,trade_pct_gdp,population
0,Brazil,1990,6149.867889,NaN,2947.732772,NaN,15.155599,149143223
1,Brazil,1991,6107.648837,6.894,432.786662,NaN,16.592078,151724256
2,Brazil,1992,5973.982856,7.086,951.962053,NaN,19.253375,154275079
3,Brazil,1993,6167.460922,6.030,1927.380790,NaN,19.599321,156794577
4,Brazil,1994,6427.169019,6.578,2075.888398,NaN,18.674760,159265006


In [63]:
df_merged = pd.merge(brazil_inequality, brazil_indicators, left_on=['country', 'year'], right_on=['country_name', 'year'])
print(df_merged.shape)
df_merged[:40]

(33, 13)


,country,year,bottom_50%,top_0.1%,top_1%,top_10%,country_name,gdp_percap_const,unemployment_pct,inflation_pct,edu_spend_pct_gdp,trade_pct_gdp,population
0,Brazil,1990,0.0912,0.1279,0.2724,0.5861,Brazil,6149.867889,NaN,2947.732772,NaN,15.155599,149143223
1,Brazil,1991,0.0912,0.1279,0.2724,0.5861,Brazil,6107.648837,6.894,432.786662,NaN,16.592078,151724256
2,Brazil,1992,0.0912,0.1279,0.2724,0.5861,Brazil,5973.982856,7.086,951.962053,NaN,19.253375,154275079
3,Brazil,1993,0.0912,0.1279,0.2724,0.5861,Brazil,6167.460922,6.030,1927.380790,NaN,19.599321,156794577
4,Brazil,1994,0.0912,0.1279,0.2724,0.5861,Brazil,6427.169019,6.578,2075.888398,NaN,18.674760,159265006
5,Brazil,1995,0.0912,0.1279,0.2724,0.5861,Brazil,6596.335727,7.092,66.007034,4.56816,16.984460,161735073
6,Brazil,1996,0.0912,0.1279,0.2724,0.5861,Brazil,6640.727007,8.035,15.757666,NaN,15.635591,164202549
7,Brazil,1997,0.0912,0.1279,0.2724,0.5861,Brazil,6764.858421,9.003,6.926713,NaN,16.576209,166661659
8,Brazil,1998,0.0912,0.1279,0.2724,0.5861,Brazil,6687.495236,10.150,3.195076,4.75666,16.438585,169159655
9,Brazil,1999,0.0912,0.1279,0.2724,0.5861,Brazil,6621.636762,11.125,4.858447,3.80062,20.982166,171641543
